# Gen Data

In [17]:
#!/usr/bin/env python
"""
make_close_tracks.py – synthetic RF-metadata generator
------------------------------------------------------
* 3 vessels start ~2 km apart off Cape Lookout, NC and head toward shore
* Injects:
    - V-222 AIS spoof at pings 10 & 25  (claimed = 444444444)
    - V-333 dark event at ping 15       (claimed = "")
    - V-111 single mis-prediction *and* spoof at ping 18
      (true_id 111111111, predicted_id 222222222, claimed_id 444444444)
"""

import pandas as pd, numpy as np
from pathlib import Path

# ---------------------- configuration ---------------------------------
rng      = np.random.default_rng(7)
dt, N    = 30, 40                               # 30-s step, 40 pings
T0       = pd.Timestamp("2025-04-22 12:00:00")
LAT_C, LON_C = 34.35, -76.52                    # Cape Lookout centre

TEMPL = {
    111111111: dict(lat0=LAT_C+0.00, lon0=LON_C-0.02, heading=285, speed=12, wifi=5, bt=3, radar=1),
    222222222: dict(lat0=LAT_C-0.01, lon0=LON_C+0.00, heading=265, speed=10, wifi=2, bt=6, radar=0),
    333333333: dict(lat0=LAT_C+0.02, lon0=LON_C+0.02, heading=300, speed=11, wifi=4, bt=4, radar=1),
}

# ---------------------- helpers ---------------------------------------
def step(lat, lon, hdg_deg, spd_kn, secs):
    """Simple great-circle step (sufficient over small distance)."""
    d_m   = spd_kn * 0.514444 * secs
    dlat  = d_m / 111_320
    dlon  = d_m / (111_320 * np.cos(np.deg2rad(lat)))
    th    = np.deg2rad(hdg_deg)
    return lat + dlat*np.cos(th), lon + dlon*np.sin(th)

rows = []
for vid, cfg in TEMPL.items():
    lat, lon = cfg['lat0'], cfg['lon0']
    for k in range(N):
        ts   = T0 + pd.Timedelta(seconds=k*dt)
        hdg  = cfg['heading'] + rng.normal(0, 2)
        lat, lon = step(lat, lon, hdg, cfg['speed'], dt)
        lat += rng.normal(0, 2e-4); lon += rng.normal(0, 2e-4)

        # ------------ default values ---------------------------------
        claimed = vid
        pred    = vid

        # ------------ scripted events --------------------------------
        if vid == 222222222 and k in (10, 25):            # AIS spoof
            claimed = 444444444

        if vid == 333333333 and k == 15:                  # dark
            claimed = ""

        if vid == 111111111 and k == 18:                  # mis-ID + spoof
            pred    = 222222222
            claimed = 444444444

        # ------------ row assembly -----------------------------------
        rows.append(dict(
            timestamp   = ts,
            lat         = round(lat, 5),
            lon         = round(lon, 5),
            wifi_ct     = int(cfg['wifi'] + rng.integers(-1, 2)),
            bt_ct       = int(cfg['bt']   + rng.integers(-1, 2)),
            radar_on    = cfg['radar'],
            vhf_ch16    = rng.integers(0, 2),
            mean_rssi   = round(-60 + rng.normal(0, 3), 1),
            true_vessel_id      = vid,
            predicted_vessel_id = pred,
            claimed_id          = claimed,
        ))

# ---------------------- DataFrame & labels ----------------------------
df = pd.DataFrame(rows).sort_values("timestamp").reset_index(drop=True)

def label_alert(row):
    if row.claimed_id == "" or pd.isna(row.claimed_id):
        return "dark"
    if int(row.true_vessel_id) == int(row.claimed_id):
        return "normal"
    return "spoof"

df["gt_alert"] = df.apply(label_alert, axis=1)

# ---------------------- write CSV -------------------------------------
out = Path("stream_demo_new.csv")
df.to_csv(out, index=False)
print(f"Wrote {out} with shore-ward tracks and one mis-spoof event.")

Wrote stream_demo_new.csv with shore-ward tracks and one mis-spoof event.


# Run Demo

In [18]:

import time, threading, pandas as pd, ipywidgets as iw
from ipyleaflet import Map, CircleMarker, Polyline, LayerGroup, WidgetControl, basemaps

# ------------------------------------------------------------------ #
# Load data
# ------------------------------------------------------------------ #
df = pd.read_csv("stream_demo_new.csv").sort_values("timestamp").reset_index(drop=True)

# ------------------------------------------------------------------ #
# helpers
# ------------------------------------------------------------------ #
def status_and_colour(true_id, claimed_id, pred_id):
    """
    Return (status_str, colour_hex)
    status = normal | spoof | dark | mis-id
    """
    # Model mistake first (light-green) ---------------------------------
    if int(pred_id) != int(true_id):
        return "mis-id", "#90EE90"          # light green

    # Regular claim checks ---------------------------------------------
    if pd.isna(claimed_id) or claimed_id == "":
        return "dark", "orange"
    if int(pred_id) == int(claimed_id):
        return "normal", "green"
    return "spoof", "red"

LABEL = {111111111: "V-111", 222222222: "V-222", 333333333: "V-333"}

# ------------------------------------------------------------------ #
# ipyleaflet map & legend
# ------------------------------------------------------------------ #
m = Map(center=(df.lat.mean(), df.lon.mean()),
        zoom=9,
        basemap=basemaps.OpenStreetMap.Mapnik)
layer = LayerGroup(); m.add(layer)

legend_html = """
<div style="background:rgba(255,255,255,0.9);padding:6px 8px;
            border:1px solid #999;border-radius:4px;font-size:11px;">
<b>Legend</b><br>
<span style='color:green;'>●</span> Normal: fingerprint matches AIS claim<br>
<span style='color:#90EE90;'>●</span> Mis-ID : prediction ≠ true vessel<br>
<span style='color:red;'>●</span> Spoof : fingerprint ≠ AIS claim<br>
<span style='color:orange;'>●</span> Dark  : no AIS claim detected
</div>
"""
m.add_control(WidgetControl(widget=iw.HTML(legend_html), position="bottomleft"))

# ------------------------------------------------------------------ #
# UI widgets
# ------------------------------------------------------------------ #
start_btn = iw.Button(description="▶ Start", button_style="success")
stop_btn  = iw.Button(description="■ Stop",  button_style="danger", disabled=True)
speed_sl  = iw.IntSlider(value=10, min=1, max=20, description="Speed ×")
ui_box    = iw.HBox([start_btn, stop_btn, speed_sl])

# ------------------------------------------------------------------ #
# playback thread (one trail per vessel)
# ------------------------------------------------------------------ #
markers, trails = {}, {}
running = False

def playback():
    global running
    running = True
    layer.clear_layers(); markers.clear(); trails.clear()

    for _, row in df.iterrows():
        if not running:
            break

        true_id = int(row.true_vessel_id)
        pred_id = true_id          # ← replace with model.predict(...) later

        status, colour = status_and_colour(true_id, row.claimed_id, pred_id)

        # trail per vessel
        if true_id not in trails:
            trails[true_id] = Polyline(locations=[], color="#6c6cff",
                                       weight=2, opacity=0.6)
            layer.add(trails[true_id])
        trails[true_id].locations.append((row.lat, row.lon))

        popup_html = (
            f"<b>{LABEL[true_id]}</b><br>{row.timestamp}<br>"
            f"True ID     : {true_id}<br>"
            f"Predicted ID: {pred_id}<br>"
            f"Claimed ID  : {row.claimed_id or '—'}<br>"
            f"Status      : {status}<hr style='margin:4px 0;'>"
            f"Wi-Fi {row.wifi_ct}   BT {row.bt_ct}<br>"
            f"RSSI {row.mean_rssi} dBm"
        )

        marker = CircleMarker(location=(row.lat, row.lon),
                              radius=5, color=colour,
                              fill_color=colour, fill_opacity=0.9)
        marker.popup = iw.HTML(popup_html)
        layer.add(marker)
        markers[true_id] = marker

        time.sleep(1.0 / speed_sl.value)

    start_btn.disabled = False
    stop_btn.disabled  = True

def on_start(_):
    start_btn.disabled, stop_btn.disabled = True, False
    threading.Thread(target=playback, daemon=True).start()

def on_stop(_):
    global running
    running = False
    stop_btn.disabled = True

start_btn.on_click(on_start)
stop_btn.on_click(on_stop)

# ------------------------------------------------------------------ #
# display
# ------------------------------------------------------------------ #
display(m, ui_box)


Map(center=[np.float64(34.36005716666667), np.float64(-76.55673683333335)], controls=(ZoomControl(options=['po…